# Day 1 실습 3: EXAONE QLoRA 파인튜닝

## 🎯 학습 목표

1. **RAFT 전처리** 이해하기 (이론)
2. **한국어 요약** 데이터로 실전 파인튜닝
3. QLoRA (4bit + LoRA) 적용

---

## 📚 구성

- **Part 1**: RAFT 전처리 (간단 설명)
- **Part 2**: 한국어 요약 파인튜닝 (메인 실습!)

## Part 1: RAFT 전처리 개념 (간단 설명)

### RAFT (Retrieval Augmented Fine-Tuning)란?

- **목적**: RAG 시스템에서 더 나은 답변 생성
- **방법**: 질문 + 정답 문서 + 오답 문서(distractor)를 함께 학습

```
입력: 질문: What is...문서: The answer is...오답 문서: Wrong info...답변:
출력: The answer is...
```

### 왜 복잡할까?

- 긴 프롬프트 (1000+ tokens)
- GPU 시간 오래 걸림
- 평가 복잡함

→ **실습에서는 더 간단한 요약 태스크 사용!**

---

RAFT 전처리는 01,02번 노트북에서 이미 배웠습니다.
여기서는 **실전 파인튜닝**에 집중하겠습니다!

---

## Part 2: 한국어 요약 파인튜닝 (메인 실습)

### 💡 왜 요약?

1. **간단**: 기사 본문 → 제목 (짧음!)
2. **빠름**: 10분 내 학습 가능
3. **명확**: 결과를 바로 확인 가능
4. **한국어**: 실제 확인 가능

### 📦 데이터셋

**daekeun-ml/naver-news-summarization-ko**
- 네이버 뉴스 기사 → 제목
- Train: 22K, Validation: 2.4K
- 예시:
  ```
  기사: "삼성전자가 오늘 신제품 발표회를 열고..."
  제목: "삼성 신제품 공개"
  ```

## 0. 패키지 설치

Colab/Kaggle에서 실행 시 아래를 실행하고 **런타임 재시작**하세요.

In [ ]:
!pip install -U bitsandbytes accelerate peft datasets transformers

# 런타임 재시작 필요!

## 1. 라이브러리 import

In [ ]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer,
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

os.environ["WANDB_DISABLED"] = "true"

print("✅ 라이브러리 로드 완료")

## 2. 데이터 로드

In [ ]:
# 한국어 뉴스 요약 데이터셋
print("📥 데이터 로드 중...")
train_dataset = load_dataset("daekeun-ml/naver-news-summarization-ko", split="train")
val_dataset = load_dataset("daekeun-ml/naver-news-summarization-ko", split="validation")

print(f"✅ 학습 데이터: {len(train_dataset):,}개")
print(f"✅ 검증 데이터: {len(val_dataset):,}개")

# 샘플 확인
sample = train_dataset[0]
print("\n📝 샘플:")
print(f"  기사: {sample['document'][:100]}...")
print(f"  요약: {sample['summary']}")

## 3. 데이터 설정

빠른 실습을 위해 샘플 수를 제한합니다.

In [ ]:
# 빠른 테스트: 100개 (5분) | 보통: 1000개 (20분) | 전체: 22K (2시간+)
MAX_TRAIN_SAMPLES = 1000
MAX_EVAL_SAMPLES = 100
MAX_LENGTH = 512

# 토큰 길이로 필터링 (짧은 샘플만 선택)
def is_valid_sample(example):
    """기사 + 요약이 MAX_LENGTH 안에 들어가는지 체크"""
    article = str(example.get("document", ""))
    summary = str(example.get("summary", ""))
    
    # 대략적인 토큰 수 추정 (한글: 1.5자당 1토큰, 영어: 0.7단어당 1토큰)
    estimated_tokens = len(article) / 1.5 + len(summary) / 1.5
    
    # 여유 있게 MAX_LENGTH의 80% 이하만 선택
    return estimated_tokens < MAX_LENGTH * 0.8

print(f"📊 원본 데이터: train={len(train_dataset):,}, val={len(val_dataset):,}")

# 필터링
train_dataset = train_dataset.filter(is_valid_sample)
val_dataset = val_dataset.filter(is_valid_sample)

print(f"✂️ 필터링 후: train={len(train_dataset):,}, val={len(val_dataset):,}")

# 샘플 선택
train_dataset = train_dataset.select(range(min(MAX_TRAIN_SAMPLES, len(train_dataset))))
val_dataset = val_dataset.select(range(min(MAX_EVAL_SAMPLES, len(val_dataset))))

print(f"✂️ 최종 샘플: train={len(train_dataset):,}, val={len(val_dataset):,}")

# 샘플 확인
sample = train_dataset[0]
print(f"\n📝 샘플:")
print(f"  기사: {sample['document'][:100]}...")
print(f"  요약: {sample['summary'][:100]}...")


## 4. 모델 & 토크나이저 로드 (4bit)

### 💡 LoRA 파라미터 설명

#### r (rank): LoRA 차원
- 낮을수록: 빠름, 성능 낮음
- 높을수록: 느림, 성능 높음
- 일반적: r=8 또는 r=16

#### alpha: 가중치 스케일
- 보통 `alpha = r × 2`
- r=8 → alpha=16

#### dropout: 과적합 방지
- 0.05 ~ 0.1 사용

---

### 🔥 QLoRA = 4bit + LoRA

- **4bit 양자화**: 메모리 1/4 감소
- **LoRA**: 전체 파라미터의 0.5~2%만 학습
- **결과**: GPU 절약 + 빠른 학습


In [ ]:
MODEL_NAME = "LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct"

print("📥 토크나이저 로드 중...")
tokenizer = AutoTokenizer.from_pretrained(
    MODEL_NAME,
    trust_remote_code=True,
    padding_side="right",
)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# 4bit 양자화 설정
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float16,
)

print("📥 모델 로드 중...")
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    trust_remote_code=True,
    torch_dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float16,
    quantization_config=quant_config,
)

print("✅ 모델 로드 완료 (4bit)")

## 5. LoRA 설정 및 적용

In [ ]:
# target_modules 자동 추출
print("🔍 첫 번째 레이어 파라미터:")
target_modules = []

for name, _ in model.named_parameters():
    if 'layers.0' in name or 'layer.0' in name or 'h.0' in name:
        if 'attn' in name.lower() and 'weight' in name:
            print(f"  {name}")
            parts = name.split('.')
            for part in parts:
                if part.endswith('_proj'):
                    if part not in target_modules:
                        target_modules.append(part)

print(f"\n✅ target_modules: {target_modules}")

lora_config = LoraConfig(
    task_type="CAUSAL_LM",
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    target_modules=target_modules,
    bias="none",
)

model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)

trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
total_params = sum(p.numel() for p in model.parameters())

print(f"\n✅ LoRA 적용: {trainable_params:,} / {total_params:,} ({trainable_params/total_params:.2%})")


### 📚 Label Masking 원리

**프롬프트는 학습 안 하고, 답변만 학습합니다.**

```python
입력: "삼성전자가 신제품을...요약: 삼성 신제품 공개"
     └───── 프롬프트 ────┘└─── 답변 ───┘

labels: [-100, -100, ..., 201, 202, 203]
        └ Loss 계산 안 함 ┘└ Loss 계산 ┘
```

| 항목 | 설명 |
|------|------|
| **프롬프트 (질문)** | 모델이 "읽는 부분". Attention에는 포함되지만, loss 계산은 안 함 |
| **답변 (정답)** | 모델이 "생성하는 부분". Loss 계산에 포함됨 |
| **이유** | 모델이 입력 복사가 아니라 "조건부 생성 능력"을 배우게 하기 위함 |

**주의**: 모델은 프롬프트를 **봅니다!** (Causal LM)
- 단지 Loss 계산만 안 함


## 6. 데이터 토크나이징

요약 형식:
```
입력: 기사 본문요약:
출력: 제목
```

In [ ]:
PROMPT_SUFFIX = "요약:"

def prepare_summarization_example(example):
    """요약 데이터를 토크나이징"""
    # 입력: 기사 본문
    article = str(example.get("document", ""))
    summary_text = str(example.get("summary", ""))
    
    # 프롬프트 생성
    prompt_text = article + PROMPT_SUFFIX
    
    # 너무 길면 자르기
    prompt_ids = tokenizer(prompt_text, add_special_tokens=False)["input_ids"]
    if len(prompt_ids) >= MAX_LENGTH - 64:
        prompt_ids = prompt_ids[:MAX_LENGTH - 64]
        prompt_text = tokenizer.decode(prompt_ids, skip_special_tokens=True) + PROMPT_SUFFIX
    
    # 전체 텍스트 (프롬프트 + 요약)
    full_text = prompt_text + " " + summary_text + tokenizer.eos_token
    
    # 토크나이징
    model_inputs = tokenizer(
        full_text,
        truncation=True,
        max_length=MAX_LENGTH,
        padding="max_length",
    )
    
    # 라벨 생성 (프롬프트 부분 마스킹)
    prompt_len = len(tokenizer(prompt_text, add_special_tokens=False)["input_ids"])
    labels = model_inputs["input_ids"].copy()
    labels[:prompt_len] = [-100] * min(prompt_len, MAX_LENGTH)
    
    # 빈 라벨이면 제외
    if all(label == -100 for label in labels):
        return None
    
    model_inputs["labels"] = labels
    return model_inputs

print("🔄 토크나이징 중...")
train_dataset = train_dataset.map(prepare_summarization_example, remove_columns=train_dataset.column_names)
train_dataset = train_dataset.filter(lambda x: x is not None)

if val_dataset is not None:
    val_dataset = val_dataset.map(prepare_summarization_example, remove_columns=val_dataset.column_names)
    val_dataset = val_dataset.filter(lambda x: x is not None)

print(f"✅ 토크나이징 완료 (학습: {len(train_dataset)}, 검증: {len(val_dataset) if val_dataset else 0})")

## 7. 학습 실행

In [ ]:
training_args = TrainingArguments(
    output_dir="./exaone_summary_lora",
    num_train_epochs=1,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=2,
    learning_rate=2e-4,
    logging_steps=50,
    save_strategy="epoch",
    report_to=[],
    remove_unused_columns=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
)

print("✅ Trainer 준비 완료")
print("🚀 파인튜닝 시작\n")

train_result = trainer.train()

print("\n✅ 파인튜닝 완료!")

## 8. 모델 저장

In [ ]:
OUTPUT_DIR = "./exaone_summary_lora_final"

print(f"💾 LoRA 어댑터 저장: {OUTPUT_DIR}")
model.save_pretrained(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)

print("\n✅ 저장 완료!")
print(f"\n📊 학습 결과:")
for key, value in train_result.metrics.items():
    print(f"   {key}: {value}")

## 10. (선택) Hugging Face 업로드

학습한 LoRA 어댑터를 Hugging Face Hub에 업로드하면 어디서든 사용 가능합니다.

In [ ]:
# Hugging Face 로그인
from huggingface_hub import login

# HF 토큰 입력 (https://huggingface.co/settings/tokens)
login(token="YOUR_HF_TOKEN_HERE")

# 업로드
repo_name = "your-username/exaone-summary-lora"  # 원하는 이름으로 변경

model.push_to_hub(repo_name)
tokenizer.push_to_hub(repo_name)

print(f"✅ 업로드 완료: https://huggingface.co/{repo_name}")
